## Social Media Engagement Analysis

### i. Introduction

#### i. 1 What's Inside this Notebook:
1. Validates the raw data
2. Controls extreme outliers
3. Stabilizes skewed metrics
4. Creates interpretable engagement metrics
5. Prepares statistics for Tableau
6. Applies robust inferential analysis

#### i. 2 Workflow:
1. Load & Validate Data
2. Outliers Handling
3. Reduce Skewness
4. Normalize by Reach (Convert engagement metrics to per 10,000 reach)
5. Engagement Quality Metrics (weighted interactions)
6. Structure Content Features
7. Segment Performance
8. Descriptive Statistics (preparing for Tableau)
9. Inferential Analysis (ANOVA, Spearman correlation)

#### i. 3 Methods Used Justification:

- Engagement data is non-normal and outlier-heavy
- Averages are misleading
- Robust, distribution-aware methods produce more trustworthy insights

### ii. Problem Identification

#### ii.1 Topic

Understanding and diagnosing engagement performance on social media posts by separating true behavioral signals from noise caused by reach imbalance, outliers, and misleading aggregate metrics.

#### ii.2 Background 

Social media performance is commonly evaluated using metrics such as likes, engagement rate, and follower growth. However, these metrics are often:

- Highly skewed due to viral content
- Strongly influenced by reach, not content quality
- Aggregated using averages, which mask typical performance

As a result, stakeholders may draw incorrect conclusions about what drives engagement and growth, leading to suboptimal content and distribution strategies.

#### ii.3 Problem Statement

Current engagement metrics do not accurately represent typical post performance and fail to distinguish between passive exposure and meaningful user interaction, making it difficult to identify reliable engagement drivers and optimization opportunities.

#### ii.4 Problem Breakdown

1. Metric Distortion by Outliers

Problem:
A small number of viral posts inflate average engagement metrics, masking typical performance.

Tableau Visualization:
- Boxplots / Distribution plots
    - Metric: engagement rate, likes, engagement depth
    - View: median, IQR, outliers
- Mean vs Median Bar Chart
    - Highlight gap between average and typical performance

Analytical Goal:
Demonstrate why averages are misleading and justify robust statistics.

2. Reach-Driven Bias

Problem:
High engagement counts are driven by exposure size rather than content effectiveness.

Tableau Visualization:
- Scatter Plot
    - X-axis: reach
    - Y-axis: likes / engagement depth
    - Color: engagement quartile
- Trend line (optional)

Analytical Goal:
Show that engagement scales with reach and does not imply engagement quality.

3. Shallow vs Meaningful Engagement

Problem:
Likes dominate engagement metrics, while active interactions are underrepresented.

Tableau Visualization:
- Stacked Bar Chart
    - Breakdown: likes vs comments + shares + saves
    - Segment by engagement quartile
- Active–Passive Ratio Distribution

Analytical Goal:
Differentiate passive consumption from meaningful engagement.

4. Content Feature Interpretability

Problem:
Raw caption length and hashtag counts are difficult to interpret and compare.

Tableau Visualization:
- Boxplots
    - Engagement depth per reach by:
        - caption bucket (short / medium / long)
        - hashtag bucket (low / optimal / high)
- Side-by-side comparison

Analytical Goal:
Identify which content structures are associated with stronger engagement quality.

5. High-Reach, Low-Engagement Risk

Problem:
Some posts achieve wide exposure but fail to generate interaction.

Tableau Visualization:
- Quadrant Chart
    - X-axis: reach tier
    - Y-axis: engagement depth per reach
    - Highlight: high-reach–low-engagement flag
- Count or % of posts per quadrant

Analytical Goal:
Expose inefficient content distribution and optimization opportunities.

6. Unclear Relationships Between Engagement Signals

Problem:
It is unclear which engagement behaviors move together reliably.

Tableau Visualization:
- Correlation Heatmap
    - Metrics: per-reach engagement metrics
    - Color: Spearman correlation coefficient
- Filter: significant relationships only

Analytical Goal
Identify consistent behavioral relationships using robust statistics.

7. Lack of Actionable Segmentation

Problem:
Global averages hide performance differences across post types.

Tableau Visualization:
- Segmented KPI Table
    - Metrics by:
    - reach tier
    - engagement quartile
- Highlight underperforming segments

Analytical Goal:
Enable targeted optimization instead of one-size-fits-all strategy.

### iii. Data Loading

#### iii.1 Data Source: Kaggle

The dataset used in this analysis is sourced from Kaggle (https://www.kaggle.com/datasets/kundanbedmutha/instagram-analytics-dataset
). It contains post-level Instagram data, including engagement metrics (likes, comments, shares, saves), exposure metrics (reach, impressions), content attributes (caption length, hashtag count), and follower growth. This dataset is used to analyze engagement patterns and performance efficiency across posts.

##### iii.2 Data Loading: Python

In [1]:
# Import Libraries

import pandas as pd
import numpy as np
import streamlit as st
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy import stats
from scipy.stats import skew, kurtosis
from scipy.stats import spearmanr
from statsmodels.stats.multitest import multipletests

In [11]:
# Data Load
df = pd.read_csv("Instagram_Analytics.csv")
df.head()

,post_id,upload_date,media_type,likes,comments,shares,saves,reach,impressions,caption_length,hashtags_count,followers_gained,traffic_source,engagement_rate,content_category
0,IG0000001,2024-11-30 09:25:22.954916,Reel,31627,7559,4530,6393,615036,1007750,1340,3,899,Home Feed,4.97,Technology
1,IG0000002,2025-08-15 09:25:22.954916,Photo,63206,3490,1680,6809,1237071,1345900,1351,20,805,Hashtags,5.59,Fitness
2,IG0000003,2025-09-11 09:25:22.954916,Reel,94373,3727,1761,8367,1127470,1305369,242,24,758,Reels Feed,8.29,Beauty
3,IG0000004,2025-09-18 09:25:22.954916,Reel,172053,7222,2875,9290,764030,897874,446,11,402,External,21.32,Music
4,IG0000005,2025-03-21 09:25:22.954916,Video,99646,2703,4444,9746,7004,495406,1905,8,155,Profile,23.52,Technology


### iv. Data Exploration and Data Cleaning

In [4]:
# Dataset Shape
df.shape

(29999, 15)

In [5]:
# Columns Name
df.columns

Index(['post_id', 'upload_date', 'media_type', 'likes', 'comments', 'shares',
       'saves', 'reach', 'impressions', 'caption_length', 'hashtags_count',
       'followers_gained', 'traffic_source', 'engagement_rate',
       'content_category'],
      dtype='object')

In [6]:
# Dataset Summary
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29999 entries, 0 to 29998
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   post_id           29999 non-null  object 
 1   upload_date       29999 non-null  object 
 2   media_type        29999 non-null  object 
 3   likes             29999 non-null  int64  
 4   comments          29999 non-null  int64  
 5   shares            29999 non-null  int64  
 6   saves             29999 non-null  int64  
 7   reach             29999 non-null  int64  
 8   impressions       29999 non-null  int64  
 9   caption_length    29999 non-null  int64  
 10  hashtags_count    29999 non-null  int64  
 11  followers_gained  29999 non-null  int64  
 12  traffic_source    29999 non-null  object 
 13  engagement_rate   29999 non-null  float64
 14  content_category  29999 non-null  object 
dtypes: float64(1), int64(9), object(5)
memory usage: 3.4+ MB


In [7]:
# Data Quality Check: Missing Value
df.isna().sum()

post_id             0
upload_date         0
media_type          0
likes               0
comments            0
shares              0
saves               0
reach               0
impressions         0
caption_length      0
hashtags_count      0
followers_gained    0
traffic_source      0
engagement_rate     0
content_category    0
dtype: int64

In [8]:
# Data Quality Check: Duplicate Records
df.duplicated().sum()

np.int64(0)

In [9]:
# Data Numerical Summary
df.describe()

,likes,comments,shares,saves,reach,impressions,caption_length,hashtags_count,followers_gained,engagement_rate
count,29999.000000,29999.000000,29999.000000,29999.000000,2.999900e+04,2.999900e+04,29999.000000,29999.000000,29999.000000,29999.000000
mean,99912.661789,5017.781426,2502.912564,7490.124637,9.965616e+05,1.246669e+06,1103.110904,14.887430,502.152805,14.419276
std,57905.967401,2888.404881,1448.320395,4352.546949,5.777906e+05,5.970924e+05,635.991308,9.005811,290.364038,29.660210
min,7.000000,0.000000,0.000000,0.000000,1.650000e+02,5.107000e+03,0.000000,0.000000,0.000000,0.120000
25%,49755.000000,2530.500000,1246.000000,3690.500000,4.936810e+05,7.469585e+05,555.000000,7.000000,251.000000,5.180000
50%,99580.000000,5042.000000,2498.000000,7483.000000,9.921810e+05,1.241026e+06,1100.000000,15.000000,501.000000,9.190000
75%,150225.000000,7518.000000,3768.000000,11294.000000,1.494798e+06,1.749264e+06,1656.000000,23.000000,755.000000,15.670000
max,200000.000000,10000.000000,5000.000000,15000.000000,1.999865e+06,2.497940e+06,2200.000000,30.000000,1000.000000,3259.820000


**Data Numerical Insight**

1. Summary
- Performance is skewed by outliers
Median engagement (9.2%) is far below the mean (14.4%), meaning typical post performance is overestimated.
- High reach, low interaction
Posts reach ~1M users on average, but engagement indicates mostly passive consumption.
- Results are driven by consistency, not growth
Content formats are standardized and deliver stable results, but follower growth remains incremental.


2. Issues
- Outlier distortion
→ Apply log transformation and winsorization; shift analysis to median and percentiles.
- Shallow engagement measurement
→ Decompose engagement into depth-based metrics (active vs passive engagement).
- Low interpretability of content features
→ Bucket caption length and hashtags; create normalized metrics per reach.


In [12]:
# Handle Extreme Outliers (Winsorization)
def winsorize(series, lower=0.01, upper=0.99):
    return series.clip(
        lower=series.quantile(lower),
        upper=series.quantile(upper)
    )

outlier_cols = [
    "likes", "comments", "shares", "saves",
    "reach", "impressions",
    "followers_gained", "engagement_rate"
]

for col in outlier_cols:
    df[f"{col}_win"] = winsorize(df[col])

In [13]:
# Log Transformation (Skew Reduction)
log_cols = [
    "likes_win", "comments_win", "shares_win",
    "saves_win", "reach_win", "impressions_win",
    "followers_gained_win"
]

for col in log_cols:
    df[f"log_{col}"] = np.log1p(df[col])

In [14]:
# Normalize Engagement Metrics: Engagement per Reach
df["likes_per_10k_reach"] = (df["likes_win"] / df["reach_win"]) * 10000
df["comments_per_10k_reach"] = (df["comments_win"] / df["reach_win"]) * 10000
df["shares_per_10k_reach"] = (df["shares_win"] / df["reach_win"]) * 10000
df["saves_per_10k_reach"] = (df["saves_win"] / df["reach_win"]) * 10000


In [15]:
# Normalize Engagement Metrics: Followers Gained Efficiency
df["followers_per_10k_reach"] = (
    df["followers_gained_win"] / df["reach_win"]
) * 10000

In [ ]:
# Engagement Depth Engineering
df["engagement_depth"] = (
    (df["likes_win"] * 1) +
    (df["comments_win"] * 3) +
    (df["shares_win"] * 4) +
    (df["saves_win"] * 2)
)

In [17]:
# Normalize depth by reach
df["engagement_depth_per_10k_reach"] = (
    df["engagement_depth"] / df["reach_win"]
) * 10000

In [18]:
# Passive vs Active Engagement Ratio
df["active_engagement"] = (
    df["comments_win"] + df["shares_win"] + df["saves_win"]
)

df["passive_engagement"] = df["likes_win"]

df["active_passive_ratio"] = (
    df["active_engagement"] / (df["passive_engagement"] + 1)
)

In [19]:
# Content Feature Bucketing: Caption Length Buckets
df["caption_bucket"] = pd.cut(
    df["caption_length"],
    bins=[-1, 500, 1200, np.inf],
    labels=["short", "medium", "long"]
)

In [20]:
# Hashtag Count Buckets
df["hashtag_bucket"] = pd.cut(
    df["hashtags_count"],
    bins=[-1, 5, 20, np.inf],
    labels=["low", "optimal", "high"]
)

In [21]:
# Performance Segmentation (Quartiles): Engagement Segments
df["engagement_quartile"] = pd.qcut(
    df["engagement_rate_win"],
    q=4,
    labels=["low", "mid-low", "mid-high", "high"]
)

In [22]:
# Performance Segmentation (Quartiles): Reach Tiers
df["reach_tier"] = pd.qcut(
    df["reach_win"],
    q=4,
    labels=["low", "medium", "high", "very_high"]
)

In [23]:
# Flag High Reach but Low Engagement
df["high_reach_low_engagement"] = (
    (df["reach_tier"].isin(["high", "very_high"])) &
    (df["engagement_quartile"].isin(["low", "mid-low"]))
)

In [24]:
# Select Final Analysis-Ready Dataset
analysis_df = df[
    [
        # Core normalized metrics
        "likes_per_10k_reach",
        "comments_per_10k_reach",
        "shares_per_10k_reach",
        "saves_per_10k_reach",
        "followers_per_10k_reach",

        # Engineered metrics
        "engagement_depth_per_10k_reach",
        "active_passive_ratio",

        # Content features
        "caption_bucket",
        "hashtag_bucket",

        # Segments
        "engagement_quartile",
        "reach_tier",
        "high_reach_low_engagement",

        # Log metrics (for stats/models)
        "log_likes_win",
        "log_comments_win",
        "log_shares_win",
        "log_saves_win",
        "log_reach_win"
    ]
]

In [25]:
analysis_df.describe(include="all")

,likes_per_10k_reach,comments_per_10k_reach,shares_per_10k_reach,saves_per_10k_reach,followers_per_10k_reach,engagement_depth_per_10k_reach,active_passive_ratio,caption_bucket,hashtag_bucket,engagement_quartile,reach_tier,high_reach_low_engagement,log_likes_win,log_comments_win,log_shares_win,log_saves_win,log_reach_win
count,29999.000000,29999.000000,29999.000000,29999.000000,29999.000000,29999.000000,29999.000000,29999,29999,29999,29999,29999,29999.000000,29999.000000,29999.000000,29999.000000,29999.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,3,4,4,2,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,long,optimal,low,low,False,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13572,14447,7509,7500,19075,NaN,NaN,NaN,NaN,NaN
mean,2796.679770,142.969935,71.734845,211.748247,14.357430,3936.025447,0.429248,NaN,NaN,NaN,NaN,NaN,11.213250,8.223623,7.531558,8.621895,13.512758
std,7459.761480,389.273955,195.742811,572.298813,39.296668,9834.122204,1.097261,NaN,NaN,NaN,NaN,NaN,0.956471,0.956881,0.941603,0.955072,0.957247
min,9.771211,0.464373,0.277614,0.765729,0.050475,57.207899,0.001886,NaN,NaN,NaN,NaN,NaN,7.568813,4.532599,4.025352,5.023881,9.910041
25%,506.460466,25.379496,12.544064,37.252844,2.510572,850.034148,0.094615,NaN,NaN,NaN,NaN,NaN,10.814886,7.836567,7.128496,8.213788,13.109647
50%,1000.281225,50.374231,25.080662,74.863401,5.013044,1401.746430,0.150996,NaN,NaN,NaN,NaN,NaN,11.508727,8.525756,7.823646,8.920523,13.807662
75%,2007.513731,100.894000,50.079052,151.536432,10.145463,2817.517906,0.302816,NaN,NaN,NaN,NaN,NaN,11.919896,8.925188,8.234565,9.332115,14.217502


In [45]:
# save file for tableau
df.to_csv(
    "ig-analytics-clean.csv",
    index=False,
    encoding="utf-8",
    sep=","
)

In [46]:
pd.read_csv("ig-analytics-clean.csv")

,post_id,upload_date,media_type,likes,comments,shares,saves,reach,impressions,caption_length,...,engagement_depth,engagement_depth_per_10k_reach,active_engagement,passive_engagement,active_passive_ratio,caption_bucket,hashtag_bucket,engagement_quartile,reach_tier,high_reach_low_engagement
0,IG0000001,2024-11-30 09:25:22.954916,Reel,31627,7559,4530,6393,615036,1007750,1340,...,85210.0,1385.447356,18482.0,31627.0,0.584356,long,low,low,medium,False
1,IG0000002,2025-08-15 09:25:22.954916,Photo,63206,3490,1680,6809,1237071,1345900,1351,...,94014.0,759.972548,11979.0,63206.0,0.189520,long,optimal,mid-low,high,True
2,IG0000003,2025-09-11 09:25:22.954916,Reel,94373,3727,1761,8367,1127470,1305369,242,...,129332.0,1147.099258,13855.0,94373.0,0.146810,short,high,mid-low,high,True
3,IG0000004,2025-09-18 09:25:22.954916,Reel,172053,7222,2875,9290,764030,897874,446,...,223799.0,2929.191262,19387.0,172053.0,0.112680,short,optimal,high,medium,False
4,IG0000005,2025-03-21 09:25:22.954916,Video,99646,2703,4444,9746,7004,495406,1905,...,145023.0,72041.429671,16893.0,99646.0,0.169528,long,optimal,high,low,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29994,IG0029995,2024-12-18 09:25:22.954916,Video,46046,8354,3847,11095,597992,820688,1254,...,108686.0,1817.515953,23296.0,46046.0,0.505918,long,optimal,mid-low,medium,False
29995,IG0029996,2025-05-05 09:25:22.954916,Carousel,67711,3266,458,12380,1908094,2218288,1427,...,104101.0,545.575847,16104.0,67711.0,0.237831,long,low,low,very_high,True
29996,IG0029997,2025-05-26 09:25:22.954916,Photo,52326,7328,3687,7619,1984066,2447893,713,...,104296.0,526.437201,18634.0,52326.0,0.356107,medium,low,low,very_high,True
29997,IG0029998,2025-08-02 09:25:22.954916,Carousel,158113,5890,2573,6329,1984709,2001092,1341,...,198733.0,1003.110803,14792.0,158113.0,0.093553,long,high,mid-low,very_high,True


### v. Analysis and Calculations

##### V. 1 Statistics Descriptive 

In [27]:
# Distributional Characteristics and Dispersion Analysis
desc = pd.DataFrame({
    "variance": df.var(numeric_only=True),
    "skewness": df.skew(numeric_only=True),
    "kurtosis": df.kurtosis(numeric_only=True)
})

desc

,variance,skewness,kurtosis
likes,3.353101e+09,0.010201,-1.203356
comments,8.342883e+06,-0.010738,-1.200813
shares,2.097632e+06,0.005787,-1.210511
saves,1.894466e+07,0.003401,-1.220095
reach,3.338420e+11,0.011810,-1.197439
impressions,3.565193e+11,0.012295,-1.063386
caption_length,4.044849e+05,-0.003024,-1.199745
hashtags_count,8.110464e+01,0.018402,-1.216393
followers_gained,8.431127e+04,-0.004265,-1.215077
engagement_rate,8.797281e+02,50.844828,4967.835606


Key Insights

- Core metrics are statistically stable
  Likes, comments, shares, saves, reach, impressions, and content features show near-zero skewness and flat distributions, making them suitable for standard statistical analysis.

- Engagement rate is unreliable as a primary KPI
  It is extremely skewed and heavy-tailed, even after winsorization, and is dominated by rare viral outliers.

- Log transformation works
  Log-scaled likes, reach, impressions, and follower gains show reduced variance and acceptable distributions, making them model-ready.

- Per-reach (normalized) metrics are highly skewed
  Metrics per 10K reach exhibit very high skewness and kurtosis, so they should be analyzed in log space or via percentiles rather than averages.

- Engagement depth is a better engagement proxy
  It is more stable than engagement rate, though its per-reach version still requires robust handling.

- Engagement is mostly passive
  Active–passive engagement ratio is heavily skewed, indicating most posts are like-driven, with truly interactive posts being rare.

- High-reach, low-engagement posts form a valid segment
  This segment is statistically consistent and suitable for targeted diagnostic and optimization analysis.


In [30]:
analysis_cols = [
    "likes_per_10k_reach",
    "comments_per_10k_reach",
    "shares_per_10k_reach",
    "saves_per_10k_reach",
    "followers_per_10k_reach",
    "engagement_depth_per_10k_reach",
    "active_passive_ratio",
    "log_likes_win",
    "log_comments_win",
    "log_shares_win",
    "log_saves_win",
    "log_reach_win"
]

In [33]:
# Central tendency
desc_ct = (
    df_numeric
    .describe()               # count, mean, std, min, 25%, 50%, 75%, max
    .T
    .reset_index()
    .rename(columns={"index": "metric"})
)

# Distributional characteristics
df_numeric = df[analysis_cols].apply(pd.to_numeric, errors="coerce")

desc_dist = (
    pd.DataFrame({
        "variance": df_numeric.var(),
        "skewness": df_numeric.skew(),
        "kurtosis": df_numeric.kurtosis()
    })
    .reset_index()
    .rename(columns={"index": "metric"})
)

# MERGE (LEFT JOIN IN PYTHON)
desc_final = desc_ct.merge(
    desc_dist,
    on="metric",
    how="left"
)

desc_final.to_csv("descriptive_statistics_tableau.csv", index=False)

#### v. 2 Statistic Inferential

In [35]:
# Statistic Inferential for Tableau
analysis_cols = [
    "likes_per_10k_reach",
    "comments_per_10k_reach",
    "shares_per_10k_reach",
    "saves_per_10k_reach",
    "followers_per_10k_reach",
    "engagement_depth_per_10k_reach",
    "active_passive_ratio"
]
  
df_numeric = df[analysis_cols + [group_col]].copy()

for col in analysis_cols:
    df_numeric[col] = pd.to_numeric(df_numeric[col], errors="coerce")

In [37]:
# ANOVA
from scipy.stats import f_oneway

anova_results = []

for metric in analysis_cols:
    groups = [
        grp[metric].dropna().values
        for _, grp in df_numeric.groupby(group_col)
        if grp[metric].notna().sum() > 10
    ]

    if len(groups) >= 2:
        f_stat, p_val = f_oneway(*groups)

        anova_results.append({
            "metric": metric,
            "test": "ANOVA",
            "grouping_variable": group_col,
            "f_statistic": f_stat,
            "p_value": p_val,
            "significant_05": p_val < 0.05
        })

anova_df = pd.DataFrame(anova_results)

C:\Users\sinta\AppData\Local\Temp\ipykernel_34056\1167706356.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for _, grp in df_numeric.groupby(group_col)


In [38]:
# Correlation Analysis
from scipy.stats import pearsonr

corr_results = []

for i, metric_x in enumerate(analysis_cols):
    for metric_y in analysis_cols[i + 1:]:
        valid = df_numeric[[metric_x, metric_y]].dropna()

        if len(valid) > 30:
            r, p_val = pearsonr(valid[metric_x], valid[metric_y])

            corr_results.append({
                "metric_x": metric_x,
                "metric_y": metric_y,
                "correlation": r,
                "p_value": p_val,
                "significant_05": p_val < 0.05
            })

corr_df = pd.DataFrame(corr_results)

In [39]:
# Group Mean Comparison Table (Effect Direction)
group_means = (
    df_numeric
    .groupby(group_col)[analysis_cols]
    .mean()
    .reset_index()
    .melt(
        id_vars=group_col,
        var_name="metric",
        value_name="group_mean"
    )
)

C:\Users\sinta\AppData\Local\Temp\ipykernel_34056\4044033057.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(group_col)[analysis_cols]


In [41]:
# Attach Mean Context to ANOVA
anova_final = anova_df.merge(
    group_means,
    on="metric",
    how="left"
)

anova_final.to_csv(
    "inferential_anova_tableau.csv",
    index=False
)

corr_df.to_csv(
    "inferential_correlation_tableau.csv",
    index=False
)

In [43]:
# Spearman Correlation

from scipy.stats import spearmanr
spearman_results = []

for i, metric_x in enumerate(analysis_cols):
    for metric_y in analysis_cols[i + 1:]:
        valid = df_numeric[[metric_x, metric_y]].dropna()

        if len(valid) > 30:
            rho, p_val = spearmanr(valid[metric_x], valid[metric_y])

            spearman_results.append({
                "metric_x": metric_x,
                "metric_y": metric_y,
                "spearman_rho": rho,
                "p_value": p_val,
                "significant_05": p_val < 0.05
            })

spearman_df = pd.DataFrame(spearman_results)

spearman_df.to_csv(
    "inferential_spearman_correlation_tableau.csv",
    index=False
)